In [48]:
import pandas as pd
import os
import sys
from importlib import reload
from sklearn.model_selection import train_test_split
import pickle

In [5]:
# append src dir to pythonpath
module_path = os.path.abspath("../..") # os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
import lib.utils as utils

Using TensorFlow backend.


In [8]:
reload(nh)

<module 'lib.notebook_helpers' from '/home/david/Documents/proj/tag-rec/src/lib/notebook_helpers.py'>

In [9]:
dataset = utils.generate_data(10000)

In [10]:
X, y, mlbx, mlby = utils.reshape_data(dataset)

In [11]:
dataset.shape

(10000, 2)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
dataset1 = utils.generate_data(17000)

In [8]:
dataset1.shape

(17000, 2)

In [9]:
dataset1.head()

,User_tags,autotags
Line_number,,
31,"[america, child, children, free, games, image,...","[arch, architecture, fort, fortification, goth..."
34,"[august, canal, church, grand, italia, italy, ...","[architecture, belfry, building, campanile, ca..."
254,"[blue, glass, house, japan, japanese, lens, li...","[abstract, blue, ferry, indoor, neon light, sh..."
315,"[bridge, gothamist, island, manhattan, nyc, sp...","[amusement park, architecture, beam, bridge, b..."
317,"[concert, d90, festival, photo, rock, scene, w...","[alley, art, black background, black, blackand..."


In [20]:
X, y, mlbx, mlby = utils.reshape_data(dataset1)

In [11]:
np.nonzero(y[0])

NameError: name 'np' is not defined

In [70]:
def generate_and_write_NCF_dataset():
    dataset = utils.generate_data(17000)
    X, y, mlbx, mlby = utils.reshape_data(dataset)
    wrap = CFDataset(X, y, mlbx, mlby)
    pickle.dump(wrap, open('NCF_dataset.pkl', 'wb'))
    

In [68]:
class CFDataset(object):
    # Wrapper for the dataset in the NCF-sense, only using the y-part of the data.
    
    def __init__(self, X, y, mlbx, mlby):
        # Sample unlabelled as negatives, 100 per item:
        self.testNegatives = [list(np.random.choice(np.where(row == 0)[0], 99)) for row in y]
        
        # Sample positives as test positives. The list in list with indices is because NCF is implemented that way.
        self.testRatings = [[i, np.random.choice(np.nonzero(row)[0])] for i, row in enumerate(y)]
        for (m, n) in self.testRatings:
            y[m, n] = 0

        self.trainMatrix = sparse.csc_matrix(y)
        self.mlby = mlby
        self.X = X
        self.mlbx = mlbx

In [71]:
generate_and_write_NCF_dataset()

In [50]:
data = CFDataset(y, mlby)

In [53]:
pickle.dump(data, open('CF_dataset.pkl', 'wb'))
x = pickle.load(open('CF_dataset.pkl', 'rb'))

In [39]:
testNegatives = [list(np.random.choice(np.where(row == 0)[0], 99)) for row in y]

In [149]:
np.random.choice(np.nonzero(y[0])[0])

446

## Start with CF

In [13]:
import implicit
from scipy import sparse
import numpy as np

In [14]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [15]:
y_train.shape

(7000, 986)

7000 "users" and 986 "items"

In [13]:
y_sparse = sparse.csr_matrix(y_train)

In [14]:
# fit takes an item_X_user-matrix
model.fit(y_sparse.transpose())

100%|██████████| 15.0/15 [00:00<00:00, 22.17it/s]


In [15]:
asd = model.recommend(1011, y_sparse)
asd

[(39, 0.35648203),
 (352, 0.22013423),
 (896, 0.16828012),
 (909, 0.13785218),
 (906, 0.13519911),
 (376, 0.12883988),
 (51, 0.115108036),
 (830, 0.10886446),
 (350, 0.100379825),
 (258, 0.087289974)]

In [16]:
y_train.shape

(7000, 986)

In [17]:
np.nonzero(y_train[0])

(array([ 59, 369, 455, 652, 761, 847]),)

In [18]:
mlby.inverse_transform(y_train)[1011]

('chicago', 'illinois', 'march', 'spring', 'united+states', 'us', 'usa')

In [19]:
cand = np.zeros((1, 986))
cand[0][112] = 1

In [20]:
np.nonzero(cand.shape)

(array([0, 1]),)

In [21]:
cand.shape

(1, 986)

In [22]:
y_train.shape

(7000, 986)

In [23]:
mlby.inverse_transform(np.reshape(test_test[545], (1, -1)))

NameError: name 'test_test' is not defined

## Further

Hur funkar det om vi endast har 1 user tag på ett item, kan vi få vettiga rekommendationer?

Hur gör de flesta papers för att testa detta? Tar man bort eller? Kolla upp detta.

Use half to start with (from epfl-flickr)

## Create an evaluation

In [84]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
class ALSEstimator(BaseEstimator, TransformerMixin):
    # from https://gist.github.com/jbochi/2e8ddcc5939e70e5368326aa034a144e#file-evaluation-ipynb
    def __init__(self, factors=50,
                       regularization=0.01,
                       iterations=15,
                       filter_seen=True,
                       show_progress=True,
                       n=3):
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.filter_seen = filter_seen
        self.show_progress = show_progress
        self.n = n

    def fit(self, X, y=None):
        self.model = implicit.als.AlternatingLeastSquares(factors=self.factors,
                                             regularization=self.regularization,
                                             iterations=self.iterations,
                                             dtype=np.float32,
                                             use_native=True)
        self.model.fit(X, show_progress=self.show_progress)
        if self.filter_seen:
            self.fit_X = X
        self._fitted = True
        return self
    
    def predict(self, X=None, y=None):
        # Check if fit had been called
        check_is_fitted(self, ['_fitted'])
        
        predictions = np.dot(self.model.item_factors, self.model.user_factors.T)
        if self.filter_seen:
            predictions[self.fit_X.nonzero()] = -99
            
        # Make predictions into top n (default is 3)
        predictions = predictions.transpose()
        tops = utils.get_top_n_tags(predictions, n=self.n)
        predictions = utils.from_keras_format(list(map(lambda x: x + 1, tops)), predictions.shape[1])
        return predictions

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
class NMSLibALSEstimator(BaseEstimator, TransformerMixin):
    # from https://gist.github.com/jbochi/2e8ddcc5939e70e5368326aa034a144e#file-evaluation-ipynb
    def __init__(self, iterations=15,
                       filter_seen=True,
                       show_progress=True):
        self.iterations = iterations
        self.filter_seen = filter_seen
        self.show_progress = show_progress

    def fit(self, X, y=None):
        self.model = implicit.approximate_als.NMSLibAlternatingLeastSquares()
        self.model.fit(X, show_progress=self.show_progress)
        if self.filter_seen:
            self.fit_X = X
        self._fitted = True
        return self
    
    def predict(self, X=None, y=None):
        # Check if fit had been called
        check_is_fitted(self, ['_fitted'])
        
        predictions = np.dot(self.model.item_factors, self.model.user_factors.T)
        if self.filter_seen:
            predictions[self.fit_X.nonzero()] = -99
            
        # Make predictions into top 3
        predictions = predictions.transpose()
        tops = utils.get_top_n_tags(predictions, n=3)
        predictions = utils.from_keras_format(list(map(lambda x: x + 1, tops)), predictions.shape[1])
        return predictions

## Test part
This part is to figure out how to do it. The proper implementation will be under the headline *Optimize*

In [19]:
test_truth = y_test.copy()
test_test = y_test.copy()
for row_index in range(y_test.shape[0]):
    nonzeros = np.nonzero(y_test[row_index])[0]
    # Set half of the non-zero elements in the row to zero. These are saved in test_truth, and will be predicted
    test_test[row_index, np.random.choice(nonzeros, int(len(nonzeros)/2), replace=False)] = 0

In [20]:
test_missing = test_truth - test_test

In [21]:
print(y_train.shape)
print(test_test.shape)
full_train = np.concatenate((y_train, test_test), axis=0)

(7000, 986)
(3000, 986)


In [22]:
model = ALSEstimator()
full_train_sparse = sparse.csr_matrix(full_train)
# fit takes an item_X_user-matrix
model.fit(full_train_sparse.transpose())

100%|██████████| 15.0/15 [00:00<00:00, 16.73it/s]


ALSEstimator(factors=50, filter_seen=True, iterations=15, regularization=0.01,
       show_progress=True)

In [27]:
model.predict().shape

(10000, 986)

In [33]:
preds_orgs = model.predict()[-3000:]

In [34]:
preds_orgs.shape

(3000, 986)

In [35]:
tops = utils.get_top_n_tags(preds_orgs, n=3)
tops

[array([679, 980, 671]),
 array([913, 511, 509]),
 array([527, 380, 351]),
 array([351, 820, 380]),
 array([935,  49, 199]),
 array([ 59, 203, 966]),
 array([ 94, 566, 388]),
 array([399, 907, 518]),
 array([527, 820, 350]),
 array([501, 350,  74]),
 array([157, 514,  81]),
 array([ 92, 120, 763]),
 array([602, 603, 915]),
 array([367, 282, 438]),
 array([762,  39, 908]),
 array([132, 586, 913]),
 array([399, 902, 907]),
 array([914,  39, 908]),
 array([509, 521, 256]),
 array([401, 894, 895]),
 array([582, 218, 743]),
 array([586, 602, 626]),
 array([739, 802, 820]),
 array([802, 501, 820]),
 array([428, 896, 472]),
 array([445, 679, 363]),
 array([918, 896, 961]),
 array([321, 980,  74]),
 array([509, 856, 375]),
 array([321, 301, 501]),
 array([64,  4,  5]),
 array([762,  39, 908]),
 array([206, 851,  74]),
 array([162, 196, 167]),
 array([927, 621, 623]),
 array([679, 671, 626]),
 array([743, 514,  81]),
 array([157, 388, 968]),
 array([468, 820, 966]),
 array([336, 935, 632]),
 ar

In [42]:
#new_preds = model.predict()[-3000:]

In [36]:
new_preds = utils.from_keras_format(list(map(lambda x: x + 1, tops)), y.shape[1])
utils.print_all_scores(test_missing, new_preds)

jaccard score: 0.12382969264088191
accuracy score: 0.009333333333333334
f1-score: 0.184111798912024
recall-score: 0.15928270042194093
precision-score: 0.21811111111111112
Hamming loss: 0.005881676808654496
Hamming score: 0.12382969264088191
---


In [372]:
test_missing.shape

(3000, 986)

In [299]:
test_missing[:10].shape

(10, 986)

In [301]:
len(mlby.inverse_transform(new_preds))

10

In [314]:
from importlib import reload

In [340]:
reload(nh)

<module 'lib.notebook_helpers' from '/home/david/Documents/proj/tag-rec/src/lib/notebook_helpers.py'>

In [47]:
utils.random_check(test_missing[:10], new_preds, mlbx, mlby)

('macro', 'olympus', 'orange', 'white')
('flowers', 'garden', 'macro')
index: 2


In [368]:
mlby.inverse_transform(full_train)[7001]

('design', 'home', 'horse', 'pretty')

In [285]:
y_test.shape

(3000, 986)

In [231]:
np.nonzero(full_train[7009])

(array([ 29, 143, 179, 269, 457, 570, 573, 634]),)

In [232]:
np.nonzero(test_truth[9])

(array([ 22,  29,  53, 143, 179, 192, 220, 269, 417, 457, 570, 573, 634,
        736, 790]),)

In [241]:
set(np.nonzero(test_truth[9])[0]) - set(np.nonzero(full_train[7009])[0])

{22, 53, 192, 220, 417, 736, 790}

In [242]:
np.nonzero(test_truth[9] - full_train[7009])

(array([ 22,  53, 192, 220, 417, 736, 790]),)

In [246]:
list(map(lambda x: x[0], asd))[:5]

[53, 192, 439, 935, 283]

## Optimize

Maximize: Recall at 3, can also try jaccard-score

The reason is that we have at least 6 user tags per item.

y_train is the full training data. We will convert this into an x and a y for this purpose, called `x_cf_train` and `y_cf_train` (`cf` stands for collaborative filtering)

In [57]:
def split_half_tags(full_data):
    x_cf_train = full_data.copy()
    for row_index in range(full_data.shape[0]):
        nonzeros = np.nonzero(full_data[row_index])[0]
        # Set half of the non-zero elements in the row to zero. These are saved in y_cf_train, and will be predicted
        x_cf_train[row_index, np.random.choice(nonzeros, int(len(nonzeros)/2), replace=False)] = 0
    # Set y_cf_train to contain the remainder of the elements.
    y_cf_train = full_data - x_cf_train
    return x_cf_train, y_cf_train

In [53]:
from sklearn.model_selection import KFold

In [54]:
from sklearn.metrics import recall_score, f1_score, jaccard_similarity_score


In [55]:
from skopt import dummy_minimize, gp_minimize, forest_minimize

In [93]:
def model_run(params=(100, 0.01)):
    factors, reg = params
    print("factors: " + str(factors) + "  reg: " + str(reg))
    np.random.seed(0)

    n_splits = 5
    scorer = recall_score
    kf = KFold(n_splits=n_splits)
    #train_val_sets = list(kf.split(y_train))
    # Rename y_train here
    score = 0
    for train_indices, val_indices in kf.split(y_train):
        train_set = y_train[train_indices]
        val_set = y_train[val_indices]
        val_x, val_y = split_half_tags(val_set)

        # The train set is concatenated with the n examples in the val_x-set. This constitutes the full train set,
        # which will be passed into the CF-model to train
        full_train = np.concatenate((train_set, val_x), axis=0)

        model = ALSEstimator(factors=factors, regularization=reg, show_progress=True, n=3)
        full_train_sparse = sparse.csr_matrix(full_train)
        model.fit(full_train_sparse.transpose())
        
        # get the last part of the predictions, which is the predictions for the n examples in the val_x-set.
        # this will be compared with the val_y-set.
        y_pred = model.predict()[-val_x.shape[0]:]
        
        score += scorer(val_y, y_pred, average='micro')
        break
    score /= n_splits
    return -score

In [94]:
model_run()

factors: 100  reg: 0.01


100%|██████████| 15.0/15 [00:01<00:00, 14.20it/s]


-0.034605867797808415

In [18]:
# Function for running the estimator version
def model_run2():
    np.random.seed(0)

    n_splits = 5
    scorer = recall_score
    kf = KFold(n_splits=n_splits)
    #train_val_sets = list(kf.split(y_train))
    # Rename y_train here
    score = 0
    for train_indices, val_indices in kf.split(y_train):
        train_set = y_train[train_indices]
        val_set = y_train[val_indices]
        val_x, val_y = split_half_tags(val_set)

        full_train = np.concatenate((train_set, val_x), axis=0)

        model = NMSLibALSEstimator(show_progress=False)
        full_train_sparse = sparse.csr_matrix(full_train)
        model.fit(full_train_sparse.transpose())

        y_pred = model.predict()[-val_y.shape[0]:]
        score += scorer(val_y, y_pred, average='micro')
    score /= n_splits
    return -score

In [93]:
model_run2()

-0.16606286837989384

In [73]:
args = (model_run, [(10, 100), (1e-4, 1e-2)])
kwargs = {"verbose": True, "random_state": 0}

In [62]:
res = dummy_minimize(*args, **kwargs)

In [63]:
res

          fun: -0.16682033946558833
    func_vals: array([-0.16462944, -0.1472439 , -0.16435353, -0.16682034, -0.15997429,
       -0.14065643, -0.15955195, -0.14422776, -0.14068927, -0.15955195,
       -0.14422776, -0.14068927, -0.15955195, -0.14422776, -0.14068927,
       -0.15955195, -0.14422776, -0.14068927, -0.15955195, -0.14422776,
       -0.14068927, -0.15955195, -0.14422776, -0.14068927, -0.15955195,
       -0.14422776, -0.14068927, -0.15955195, -0.14422776, -0.14068927,
       -0.15955195, -0.14422776, -0.14068927, -0.15955195, -0.14422776,
       -0.14068927, -0.15955195, -0.14422776, -0.14068927, -0.15955195,
       -0.14422776, -0.14068927, -0.15955195, -0.14422776, -0.14068927,
       -0.15955195, -0.14422776, -0.14068927, -0.15955195, -0.14422776,
       -0.14068927, -0.15955195, -0.14422776, -0.14068927, -0.15955195,
       -0.14422776, -0.14068927, -0.15955195, -0.14422776, -0.14068927,
       -0.15955195, -0.14422776, -0.14068927, -0.15955195, -0.14422776,
       -0.140

In [67]:
res_gp = gp_minimize(*args, **kwargs, n_calls=30)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 9.5298
Function value obtained: -0.1632
Current minimum: -0.1632
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 10.4332
Function value obtained: -0.1674
Current minimum: -0.1674
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 8.6186
Function value obtained: -0.1649
Current minimum: -0.1674
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 7.6696
Function value obtained: -0.1548
Current minimum: -0.1674
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 7.3053
Function value obtained: -0.1540
Current minimum: -0.1674
Iteration No: 6 started.

/home/david/Documents/proj/proj/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 10.6430
Function value obtained: -0.1661
Current minimum: -0.1674
Iteration No: 14 started. Searching for the next optimal point.


/home/david/Documents/proj/proj/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 10.6801
Function value obtained: -0.1661
Current minimum: -0.1674
Iteration No: 15 started. Searching for the next optimal point.
Iteration No: 15 ended. Search finished for the next optimal point.
Time taken: 10.2475
Function value obtained: -0.1660
Current minimum: -0.1674
Iteration No: 16 started. Searching for the next optimal point.
Iteration No: 16 ended. Search finished for the next optimal point.
Time taken: 10.5141
Function value obtained: -0.1661
Current minimum: -0.1674
Iteration No: 17 started. Searching for the next optimal point.
Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 10.1423
Function value obtained: -0.1649
Current minimum: -0.1674
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 10.7149
Function value obtained: -0.1661
Current minimum: -0.1674
Iteration No: 

...

/home/david/Documents/proj/proj/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 11.1903
Function value obtained: -0.1661
Current minimum: -0.1674
Iteration No: 64 started. Searching for the next optimal point.


/home/david/Documents/proj/proj/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 11.2093
Function value obtained: -0.1661
Current minimum: -0.1674
Iteration No: 65 started. Searching for the next optimal point.


KeyboardInterrupt: 

In [74]:
res_forest = forest_minimize(*args, **kwargs, n_calls=20)

Iteration No: 1 started. Evaluating function at random point.
factors: 57  reg: 0.007180374727086955
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 8.8878
Function value obtained: -0.1591
Current minimum: -0.1591
Iteration No: 2 started. Evaluating function at random point.
factors: 77  reg: 0.008593661614465294
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 9.4533
Function value obtained: -0.1656
Current minimum: -0.1656
Iteration No: 3 started. Evaluating function at random point.
factors: 19  reg: 0.006273280598181128
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 7.2060
Function value obtained: -0.1343
Current minimum: -0.1656
Iteration No: 4 started. Evaluating function at random point.
factors: 46  reg: 0.003045592604790276
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 8.0003
Function value obtained: -0.1570
Current minimum: -0.1656
Iteration No: 5 started. Evaluating function at random point.
fa

In [75]:
res_forest

          fun: -0.16561284350915723
    func_vals: array([-0.15905585, -0.16561284, -0.13429457, -0.15700916, -0.16503507,
       -0.16498886, -0.15861365, -0.16480048, -0.15360956, -0.13429457,
       -0.16276125, -0.16508829, -0.16519118, -0.16508829, -0.15642602,
       -0.15394532, -0.16436156, -0.15630022, -0.16439566, -0.12889466])
       models: [ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=3,
          min_samples_split=2, min_variance=0.0,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
          oob_score=False, random_state=209652396, verbose=0,
          warm_start=False), ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=3,
          min_samples_split=2, min_variance=0.0,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
          oob_score=False, random_s